In [1]:
import os

In [2]:
# Confirm the current working directory
%pwd

'D:\\OneDrive\\Documents\\PERSONAL\\PERSONAL DEVELOPMENT\\DATA SCIENCE\\Personal Project Portfolio\\PP00015_MLOps_End-to-End-ML-Application-Project_MLflow\\research'

In [3]:
# Change the directory to one level below the current working directory
os.chdir('../')

In [4]:
# Confirm the change of directory
%pwd

'D:\\OneDrive\\Documents\\PERSONAL\\PERSONAL DEVELOPMENT\\DATA SCIENCE\\Personal Project Portfolio\\PP00015_MLOps_End-to-End-ML-Application-Project_MLflow'

In [27]:
# Creating the entity
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [56]:
# Import the needed contants and methods
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [60]:
config_filepath = CONFIG_FILE_PATH
print(config_filepath)
print(PARAMS_FILE_PATH)
params_filepath = PARAMS_FILE_PATH
# schema_filepath = SCHEMA_FILE_PATH):

#config = read_yaml(config_filepath)
params = read_yaml(params_filepath)
# self.schema = read_yaml(schema_filepath)

# create_directories([self.config.artifacts_root])

config\config.yaml
params.yaml
[2023-10-01 17:46:43,860: INFO: common: yaml file: params.yaml loaded successfully]


ValueError: yaml file is empty

In [79]:
# Define the configuration manager
class ConfigurationManager:
    
    def __int__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH, schema_filepath=SCHEMA_FILE_PATH):
#         self.config = read_yaml(config_filepath)
#         self.params = read_yaml(params_filepath)
#         self.schema = read_yaml(schema_filepath)
        self.attr = 2
#         create_directories([self.config.artifacts_root])




In [80]:
config = ConfigurationManager()

In [82]:
config.attr

AttributeError: 'ConfigurationManager' object has no attribute 'attr'

In [ ]:
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        return data_ingestion_config

In [62]:
import os
import urllib.request as request
import zipfile
from mlProject import logger
from mlProject.utils.common import get_size

In [63]:
# Update components

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config  
    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
            url = self.config.source_URL,
            filename = self.config.local_data_file
            )
            logger.info(f'{filename} download! with the following info: \n{headers}')
        else:
            logger.info(f'File already exists of size: {get_size(Path(self.config.local_data_file))}')
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.Zipfile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [64]:
# Update pipeline
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

AttributeError: 'ConfigurationManager' object has no attribute 'config'